In [40]:
import tkinter as tk
from tkinter import ttk
import random

# 定義桌子的大小和數量
tables = [None] * 12  # 12個6人桌

# 定義初始客人數量
num_initial_guests = 12 * 6  # 初始狀態為全桌坐滿，每個桌子可坐6人
initial_groups = []
group_id = 1

# 填滿所有桌子
for i in range(len(tables)):
    table = []
    remaining_seats = 6
    while remaining_seats > 0:
        group_size = random.randint(1, min(4, remaining_seats))  # 每組客人1到4人
        leave_time = random.randint(2, 7)  # 隨機離開時間
        table.append((group_id, group_size, 0, leave_time))
        initial_groups.append((group_id, group_size, 0, leave_time))
        group_id += 1
        remaining_seats -= group_size
    tables[i] = table

# 初始化後續到達的客人，每個時間點2到3組
arrival_times = []
group_sizes = []
current_time = 0
while len(arrival_times) < num_initial_guests * 2:  # 總共到達的組數
    current_time += 1  # 每個時間點都有2到3組客人到達
    for _ in range(random.randint(4,6)):
        arrival_times.append(current_time)
        group_sizes.append(random.randint(1, 4))

# 合併初始組和後續組
arrival_times = [0] * len(initial_groups) + arrival_times
group_sizes = [group[1] for group in initial_groups] + group_sizes

# 等候行列
waiting_queue = []
# 入座後的組別及其離開時間
seated_groups = initial_groups[:]

# 初始化Tkinter窗口
root = tk.Tk()
root.title('餐廳座位安排模擬')

# 畫布來顯示桌子狀態
canvas = tk.Canvas(root, width=800, height=800)
canvas.pack(side=tk.LEFT)

# 顯示等候行列和等待時間總和
waiting_label = tk.Label(root, text="等待中：", justify=tk.LEFT)
waiting_label.pack(side=tk.RIGHT, padx=10)

# 顯示當前時間點變化
change_log_label = tk.Label(root, text="座位安排：\n", justify=tk.LEFT)
change_log_label.pack(side=tk.RIGHT, padx=10, pady=10)

# 顯示預計等待時間
expected_wait_time_label = tk.Label(root, text="預計等待時間：\n", justify=tk.LEFT)
expected_wait_time_label.pack(side=tk.RIGHT, padx=10, pady=10)

# 顯示加權等待時間總和
total_wait_time_label = tk.Label(root, text="等待時間總和：0", justify=tk.LEFT)
total_wait_time_label.pack(side=tk.BOTTOM, pady=10)

# 顯示當前時間點
time_label = tk.Label(root, text=f"時間：0", justify=tk.LEFT)
time_label.pack(side=tk.TOP, pady=10)

# 控制時間點的按鈕
button_frame = tk.Frame(root)
button_frame.pack(side=tk.BOTTOM, pady=10)
next_button = ttk.Button(button_frame, text="下一個時間點", command=lambda: change_time(1))
next_button.pack(side=tk.RIGHT, padx=10, pady=6)

# 當前時間點
current_time = 0

# 模擬座位安排
def simulate():
    global current_time, waiting_queue, tables, seated_groups, changed_tables, changed_seats
    # 清空畫布
    canvas.delete("all")

    # 顯示桌子狀態
    y_offset = 50
    for i, table in enumerate(tables):
        x_offset = 50 + (i % 3) * 200  # 調整 x 間距
        if i % 3 == 0 and i != 0:
            y_offset += 180  # 調整 y 間距
        table_text = f"第{i+1}桌：\n"
        seat_count = 0
        if table is None or len(table) == 0:
            table_text += "空"
            fill_color = "light yellow"
            for j in range(6):
                seat_x = x_offset + (j % 3) * 50  # 調整座位 x 間距
                seat_y = y_offset + (j // 3) * 40  # 調整座位 y 間距
                canvas.create_rectangle(seat_x, seat_y, seat_x + 50, seat_y + 40, fill=fill_color)
                canvas.create_text(seat_x + 25, seat_y + 20, text="空")
        else:
            for group in table:
                table_text += f"第{group[0]}組 {group[1]}人\n"
                for _ in range(group[1]):
                    seat_x = x_offset + (seat_count % 3) * 50  # 調整座位 x 間距
                    seat_y = y_offset + (seat_count // 3) * 40  # 調整座位 y 間距
                    if (i, seat_count) in changed_seats:
                        fill_color = "light green"
                    else:
                        fill_color = "white"
                    canvas.create_rectangle(seat_x, seat_y, seat_x + 50, seat_y + 40, fill=fill_color)
                    canvas.create_text(seat_x + 25, seat_y + 20, text=f"{group[0]}")
                    seat_count += 1
            for j in range(seat_count, 6):
                seat_x = x_offset + (j % 3) * 50  # 調整座位 x 間距
                seat_y = y_offset + (j // 3) * 40  # 調整座位 y 間距
                canvas.create_rectangle(seat_x, seat_y, seat_x + 50, seat_y + 40, fill="light yellow")
                canvas.create_text(seat_x + 25, seat_y + 20, text="空")
        canvas.create_text(x_offset + 75, y_offset + 140, text=table_text, font=('Arial', 10))  # 調整桌號文字位置

    # 顯示等候行列和等待時間總和
    waiting_text = "等待中的組別：\n"
    total_wait_time = 0
    # 計算已經入座組別的加權等待時間總和
    for group in seated_groups:
        if group[2] > 0:  # 只計算非初始組
            total_wait_time += (group[2] - group[4]) * group[1]
    # 計算未入座組別的加權等待時間總和
    for group in waiting_queue:
        if group[2] <= current_time:
            waiting_text += f"第 {group[0]}組 ({group[1]}人) - 等待時間 {current_time - group[2]}\n"
            total_wait_time += (current_time - group[2]) * group[1]
    waiting_label.config(text=waiting_text)
    total_wait_time_label.config(text=f"等待時間總和：{total_wait_time}", font=16)
    time_label.config(text=f"現在時間：{current_time}")

    # 顯示預計等待時間
    expected_wait_times = calculate_expected_wait_times()
    expected_wait_text = "預計等待時間：\n"
    for size, wait_time in expected_wait_times.items():
        expected_wait_text += f"{size} 人：{wait_time} 分鐘\n"
    expected_wait_time_label.config(text=expected_wait_text)

    # 更新時間點變化日誌
    update_change_log()

# 時間點變更函數
def change_time(delta):
    global current_time, changed_tables, changed_seats
    current_time += delta
    current_time = max(0, current_time)
    changed_tables, changed_seats = update_seating()
    simulate()

# 更新座位安排和等候行列
def update_seating():
    global current_time, waiting_queue, tables, seated_groups

    changed_tables = set()
    changed_seats = set()

    # 新的組別到達
    for i in range(len(arrival_times)):
        if arrival_times[i] == current_time:
            group_id = i + 1 + len(initial_groups)  # 確保新到的組別編號從初始組之後開始
            group_size = group_sizes[i]
            arrival_time = arrival_times[i]
            waiting_queue.append((group_id, group_size, arrival_time))

    # 移除應該離開的組別
    for i in range(len(tables)):
        if tables[i]:
            new_table = []
            seat_count = 0
            for group in tables[i]:
                if group[3] == current_time:
                    for _ in range(group[1]):
                        changed_seats.add((i, seat_count))
                        seat_count += 1
                    changed_tables.add(i)
                else:
                    new_table.append(group)
                    seat_count += group[1]
            tables[i] = new_table if new_table else None

    # 嘗試將等候行列中的組別安排入座
    new_waiting_queue = []
    for group in waiting_queue:
        group_id, group_size, arrival_time = group
        seated = False
        for i in range(len(tables)):
            if tables[i] is None:
                tables[i] = [(group_id, group_size, current_time, current_time + random.randint(5, 8), arrival_time)]
                changed_tables.add(i)
                for j in range(group_size):
                    changed_seats.add((i, j))
                seated_groups.append(tables[i][0])
                seated = True
                break
            else:
                total_seats = sum(g[1] for g in tables[i])
                if total_seats + group_size <= 6:
                    dining_time = random.randint(5, 8)  # 用餐時間為5~8分鐘
                    tables[i].append((group_id, group_size, current_time, current_time + dining_time, arrival_time))
                    seated_groups.append(tables[i][-1])
                    for j in range(total_seats, total_seats + group_size):
                        changed_seats.add((i, j))
                    changed_tables.add(i)
                    seated = True
                    break
        if not seated:
            new_waiting_queue.append(group)
    waiting_queue[:] = new_waiting_queue

    return changed_tables, changed_seats

# 計算預計等待時間
def calculate_expected_wait_times():
    wait_times = {1: 0, 2: 0, 3: 0, 4: 0}
    counts = {1: 0, 2: 0, 3: 0, 4: 0}

    for table in tables:
        if table:
            for group in table:
                size = group[1]
                wait_times[size] += group[3] - current_time
                counts[size] += 1

    expected_wait_times = {}
    for size in wait_times:
        if counts[size] > 0:
            expected_wait_times[size] = wait_times[size] // counts[size]
        else:
            expected_wait_times[size] = 0

    return expected_wait_times

# 更新時間點變化日誌
def update_change_log():
    global change_log_label, changed_tables, changed_seats
    change_log_text = "當前時間點變化：\n"
    for table_index in changed_tables:
        groups = tables[table_index]
        if groups:
            for group in groups:
                if group[2] == current_time:  # 只顯示新入座的組別
                    change_log_text += f"第{group[0]}組（{group[1]}人）入座第{table_index+1}桌\n"
    change_log_label.config(text=change_log_text)

# 初始化模擬
changed_tables = set()
changed_seats = set()
simulate()

# 進入Tkinter主循環
root.mainloop()
